In [1]:
import os
import random
import gc
import numpy as np
from glob import glob

from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input

random.seed(34)

import sys

# Скачайте полные данные отсюда https://www.kaggle.com/c/dogs-vs-cats/data (примерно 820мб)
# У вас должно быть две папки train и test. Однако в данном задании мы не будем использвать test выборку
# Функцию для чтения данных мы реализовали за вас

from tqdm.notebook import tqdm

def read_dataset():
    X = np.array([])
    y = np.array([])

    image_paths_list = glob(os.path.join('data', 'train', '*.jpg'))
    image_paths_list = random.sample(image_paths_list, 5000)
    np.random.shuffle(image_paths_list)

    for image_path_batch in tqdm(np.split(np.array(image_paths_list), 5000 / 1000), desc='Batches'):
        x_batch = []
        y_batch = []
        
        for image_path in tqdm(image_path_batch, desc='Current batch', leave=False):
            image_name_parts = os.path.basename(image_path).split('.')
            label = image_name_parts[0] if len(image_name_parts) == 3 else None
            
            if label:
                y_batch.append(int(label == 'cat'))
            
            x = image.img_to_array(image.load_img(image_path, target_size=(224, 224)))
            x_batch.append(x)
        
        x_batch = preprocess_input(np.array(x_batch))
        y_batch = np.array(y_batch)

        if X.size == 0:
          X = x_batch
          y = y_batch
        else:
          X = np.concatenate([X, x_batch], axis=0)
          y = np.concatenate([y, y_batch], axis=0)
    
    return X, y

In [2]:
# Используйте функцию read_dataset чтобы получить обучающую выборку
features, target = read_dataset()

# Проверьте размерности загруженных данных
print(features.shape, target.shape)


Batches:   0%|          | 0/5 [00:00<?, ?it/s]

Current batch:   0%|          | 0/1000 [00:00<?, ?it/s]

Current batch:   0%|          | 0/1000 [00:00<?, ?it/s]

Current batch:   0%|          | 0/1000 [00:00<?, ?it/s]

Current batch:   0%|          | 0/1000 [00:00<?, ?it/s]

Current batch:   0%|          | 0/1000 [00:00<?, ?it/s]

(5000, 224, 224, 3) (5000,)


Перепишите код read_dataset() таким образом чтобы читать изображения пачками по 1000 и сразу применять предобученный ResNet на них, сохраняя только полученные вектора как множество X. Примените эту функцию на всём датасете 25000 изображений.

In [3]:
from sklearn.model_selection import train_test_split

# с помощью функции train_test_split поделите выборку на train и test в отношении 70/30
# В качестве признаков используйте эмбеддинги, полученные на предыдущем шаге

features_train, features_test, target_train, target_test = train_test_split(features, target, train_size=0.7, test_size=0.3, random_state=42)
print(len(features_train))

del features
gc.collect()

3500


0

In [4]:
from keras.applications.resnet50 import ResNet50

# Создаём модель для получения эмбеддингов
model = ResNet50(weights='imagenet')

# Используя объект model постройте эмбеддинги для обучающей выборки
embds_train = model.predict(features_train)
embds_test = model.predict(features_test)




Создайте объект XGBClassifier со стандартными параметрами

https://xgboost.readthedocs.io/en/latest/python/python_api.html#xgboost.XGBClassifier

Обучите его

In [5]:
from xgboost import XGBClassifier

# Получите предсказания на тестовом и на обучающем множестве
clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
clf.fit(embds_train, target_train)

predicted = clf.predict(embds_test)


In [6]:
from sklearn.metrics import accuracy_score

# С помощью функции accuracy_score оцените результаты
accuracy_score(target_test, predicted)

0.9766666666666667

Почитайте про параметры XGBoost здесь: https://sites.google.com/view/lauraepp/parameters
Сделайте тюнинг гиперпараметров для XGBoost:
1. Подберите максимальный num_iterations при котором скор на валидации близок к максимальному, а скорость обучения ещё приемлимая
2. Для заданного num_iterations подберите подходящий learning rate
3. С помощью библиотеки hyperopt подберите остальные гиперпараметры при фиксимрованных num_iterations и learning_rate. Не подбирайте параметр early_stopping

In [10]:
clf_tuned = XGBClassifier(n_estimators=505, learning_rate=0.02, use_label_encoder=False, eval_metric='logloss')
eval_set = [(embds_train, target_train), (embds_test, target_test)]
clf_tuned.fit(embds_train, target_train, eval_set=eval_set, verbose=False)

preds_tuned = clf_tuned.predict(embds_test)

accuracy_score(target_test, preds_tuned)

0.9773333333333334

In [12]:
from sklearn.model_selection import cross_val_score
 
from hyperopt import hp, tpe
from hyperopt.fmin import fmin

def hyperopt_xgb_score(params):
    clf = XGBClassifier(**params)
    current_score = cross_val_score(clf, embds_train, target_train, cv=3).mean()
    print(current_score, params)
    return -current_score
 
space = {
    'n_estimators': 505,
    'learning_rate': 0.02,
    'use_label_encoder': False,
    'eval_metric': 'logloss',
    'eta': hp.quniform('eta', 0.025, 0.5, 0.025),
    'max_depth':  hp.choice('max_depth', np.arange(1, 14, dtype=int)),
    'min_child_weight': hp.quniform('min_child_weight', 1, 6, 1),
    'subsample': hp.quniform('subsample', 0.5, 1, 0.05),
    'gamma': hp.quniform('gamma', 0.5, 1, 0.05),
    'colsample_bytree': hp.quniform('colsample_bytree', 0.5, 1, 0.05),
}
 
best = fmin(fn=hyperopt_xgb_score, space=space, algo=tpe.suggest, max_evals=10)
print('best:')
print(best)

0.980286201002115                                     
{'colsample_bytree': 0.9500000000000001, 'eta': 0.47500000000000003, 'eval_metric': 'logloss', 'gamma': 0.9, 'learning_rate': 0.02, 'max_depth': 5, 'min_child_weight': 1.0, 'n_estimators': 505, 'subsample': 0.7000000000000001, 'use_label_encoder': False}
0.9791426904246423                                                              
{'colsample_bytree': 0.75, 'eta': 0.35000000000000003, 'eval_metric': 'logloss', 'gamma': 0.5, 'learning_rate': 0.02, 'max_depth': 12, 'min_child_weight': 5.0, 'n_estimators': 505, 'subsample': 1.0, 'use_label_encoder': False}
0.9785706901679158                                                              
{'colsample_bytree': 0.8, 'eta': 0.2, 'eval_metric': 'logloss', 'gamma': 0.9, 'learning_rate': 0.02, 'max_depth': 8, 'min_child_weight': 5.0, 'n_estimators': 505, 'subsample': 0.7000000000000001, 'use_label_encoder': False}
0.9782843225875674                                                           

Оцените важность признаков с помощью поля feature_importances_

Пользуясь списком https://gist.github.com/yrevar/942d3a0ac09ec9e5eb3a посмотрите, какие классы оказались наиболее важными (со значениями importance больше 0)
    
Изменяйте параметры, чтобы оставить только действительно важные признаки

In [13]:
clf_best = XGBClassifier(n_estimators=505, learning_rate=0.02,
                         colsample_bytree=0.55, eta=0.35000000000000003,
                         gamma=1.0, max_depth=5, min_child_weight=3.0, subsample=0.55,
                         use_label_encoder=False, eval_metric='logloss')
clf_best.fit(embds_train, target_train, eval_set=eval_set, verbose=False)


list = open('imagenet1000_clsidx_to_labels.txt', 'r') 
labels = []
 
for line in list.readlines():
    labels.append(line)

all_important_features = np.where(clf_tuned.feature_importances_ > 0)

for important_features in all_important_features:
    for i in important_features:
        print("Important classes: {}".format(labels[i]))

top_ten_features = np.argpartition(clf_tuned.feature_importances_, -10)[-10:]
print('Importance of top 10 features: ', clf_tuned.feature_importances_[top_ten_features])

k = 1
for top in top_ten_features: 
    print("Top {} important features indices: {}".format(k, labels[top]))
    k += 1

Important classes: {0: 'tench, Tinca tinca',

Important classes:  3: 'tiger shark, Galeocerdo cuvieri',

Important classes:  4: 'hammerhead, hammerhead shark',

Important classes:  5: 'electric ray, crampfish, numbfish, torpedo',

Important classes:  7: 'cock',

Important classes:  8: 'hen',

Important classes:  9: 'ostrich, Struthio camelus',

Important classes:  11: 'goldfinch, Carduelis carduelis',

Important classes:  14: 'indigo bunting, indigo finch, indigo bird, Passerina cyanea',

Important classes:  15: 'robin, American robin, Turdus migratorius',

Important classes:  16: 'bulbul',

Important classes:  17: 'jay',

Important classes:  18: 'magpie',

Important classes:  19: 'chickadee',

Important classes:  21: 'kite',

Important classes:  22: 'bald eagle, American eagle, Haliaeetus leucocephalus',

Important classes:  23: 'vulture',

Important classes:  24: 'great grey owl, great gray owl, Strix nebulosa',

Important classes:  25: 'European fire salamander, Salamandra salamandr

Importance of top 10 features:  [0.00783377 0.00857336 0.01007659 0.01461885 0.01166485 0.0114389
 0.01057868 0.01106363 0.11387297 0.14691098]
Top 1 important features indices:  284: 'Siamese cat, Siamese',

Top 2 important features indices:  907: 'wine bottle',

Top 3 important features indices:  796: 'ski mask',

Top 4 important features indices:  283: 'Persian cat',

Top 5 important features indices:  180: 'American Staffordshire terrier, Staffordshire terrier, American pit bull terrier, pit bull terrier',

Top 6 important features indices:  200: 'Tibetan terrier, chrysanthemum dog',

Top 7 important features indices:  768: 'rugby ball',

Top 8 important features indices:  45: 'Gila monster, Heloderma suspectum',

Top 9 important features indices:  285: 'Egyptian cat',

Top 10 important features indices:  281: 'tabby, tabby cat',

